### Questions: Which time of the year is the busiest and how does it affect the prices?
    

In [1]:
import os
import sys
from pathlib import Path

import chart_studio
import chart_studio.plotly as py
import pandas as pd
import plotly.graph_objects as go
from dotenv import load_dotenv
from plotly.subplots import make_subplots

%matplotlib inline

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)




In [2]:
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

username = os.environ.get('PLOTLY_USER')
api_key = os.environ.get('PLOTLY_TOKEN')

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


In [ ]:
def plot_multi(df, x, y1, y2):
    """Plot a chart with 2 y axes.
    
    Args:
        df: dataframe
        x: name of the x column
        y1: name of the 1st y column
        y2: name of the 2nd y column

    Returns:
        figure

    """
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df[x], y=df[y1], name=y1),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df[x], y=df[y2], name=y2),
        secondary_y=True,
    )


    # Set x-axis title
    fig.update_xaxes(title_text=x)

    # Set y-axes titles
    fig.update_yaxes(title_text=y1, secondary_y=False)
    fig.update_yaxes(title_text=y2, secondary_y=True)
    
    

    fig.show()
    return fig

In [ ]:
df = pd.read_csv('../data/seattle/calendar.csv')
#Quick glimpse at the data
df.head()

In [ ]:
#Size of the data set
df.shape

In [ ]:
#Looks like we have to transform the columns to the appropriate data types
df.dtypes

In [ ]:
#Check if any values are missing. Luckily only some prices are missing due to availability
df.isnull().sum()

In [ ]:
#Check if there are rows with price and no availability or vice versa
df[(df['available']=='t') & pd.isna(df['price'])]

In [ ]:
df[(df['available']=='f') & (pd.isna(df['price'])==False)]

In [ ]:
#Check how many unique values in each column
df.nunique()

In [ ]:
#Check if every listing has the same number of dates.
df.groupby('listing_id')['date'].count().unique()

Looks like the data set is very clean and well-prepared. Now we should convert the 'date', 'available' and 'price' columns to the appropriate data types.

In [ ]:
bool_mapping = {'t':True, 'f':False}

df['available'] = df['available'].map(bool_mapping) 
df['date'] = pd.to_datetime(df['date'])
df['price'] = df['price'].replace('[$,]', '', regex=True).astype(float)

df.dtypes

Now we are ready to do some analysis

In [ ]:
#Let's take a look at daily averaged values data.
group_col = 'date'
agg_col1 = 'price'
agg_col2 = 'available'

daily_df = (df.groupby(group_col).agg({agg_col1: 'mean',
                                       agg_col2: 'mean'}))

daily_df = daily_df.reset_index()
x = 'Date'
y_1 = 'Average Price'
y_2 = 'Available Listings'
daily_df = daily_df.rename(columns={group_col: x, agg_col1: y_1, agg_col2: y_2})

fig = plot_multi(daily_df, x, y_1, y_2)
py.plot(fig, filename='daily_price_and_availability', auto_open = False)


This looks odd. Looks like a bunch of listings become unavailable approx. on the 1st of April and on the 1st of July. The data file "listings" has a column "last_scraped" with the value Jan 4th 2016. I assume that the file "calendar" was scraped at the same date. Considering this the data makes sense, since Airbnb has an option for the hosts to open a booking calendar for 3, 6 or 12 months in the future. So those listings are not booked, they are just not available for booking at the beginning of the year.

The fact, that the booking calendar was scraped in the beginning of the year makes the pricing and availability data less interesting since those will change drastically as the year passes.

Another thing that strikes the eye is that the price oscillates on a weekly basis. I assume that the prices are higher on the weekend.

The average price is higher in the summer than in the winter. However, it does not come down to the same level by the end of the year. I would not read too much into it because it is not the final data. Airbnb also has an option for the hosts to adjust the price automatically based on current demand.

In [ ]:
#Let's exclude the listing with partial availability calendars and look at the data again.

#Check which 2 dates have the 2 big drops in availability.
(df[df['available']==True].groupby('listing_id')
                          .agg({'date':'max'})
                          .groupby('date')
                          .agg(count=('date','count'))
                          .sort_values(by='count', ascending=False)
                          .head()
)

In [ ]:
#Get the latest available date for each listing
last_avail_df = (df[df['available']==True].groupby('listing_id').agg({'date':'max'}))

#Listings which are not limited to 3 or 6 months availability
last_avail_eoy_df = last_avail_df[(last_avail_df['date']!= '2016-04-01') & (last_avail_df['date']!= '2016-06-30')]

eoy_listings = last_avail_eoy_df.index.to_list()

#Exclude the listing which are limited to 3 or 6 months availability and plot a daily chart.

eoy_df = df[df['listing_id'].isin(eoy_listings)]


In [ ]:
#Calculate daily averages

daily_eoy_df = (eoy_df.groupby(group_col).agg({agg_col1: 'mean', agg_col2: 'mean'}))

daily_eoy_df = daily_eoy_df.reset_index()
daily_eoy_df = daily_eoy_df.rename(columns={group_col: x, agg_col1: y_1, agg_col2: y_2})

fig = plot_multi(daily_eoy_df, x, y_1, y_2)
py.plot(fig, filename='daily_price_and_availability_fixed', auto_open = False)


It looks much smoother now. The availability plot still does not tell us which time of the year is the busiest since the data was scraped in the beginning of the year. The plot goes more or less asymptotically against 1 with a slight dip in the summer months.

One can also see that the availability oscillates on the weekly basis but the further in to the future the smaller this oscillation becomes until it is not visible anymore. I assume that in the near future there are more short term bookings on the weekends which is not the case if we look at the data 6+ months in the future.

In [ ]:
group_col = df['date'].dt.weekday

weekday_df = (df.groupby(group_col).agg({agg_col1: 'mean', agg_col2: 'mean'}))

weekday_df = weekday_df.reset_index()
x = 'Weekday'
weekday_df[x] = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
weekday_df = weekday_df.rename(columns={agg_col1: y_1, agg_col2: y_2})

fig = plot_multi(weekday_df, x, y_1, y_2)
py.plot(fig, filename='dow_price_and_availability', auto_open = False)


As expected the prices are highest and availability is the lowest on the weekends. This is just a qualitative check since the data is not final.

In [ ]:
#Additionally we can look at the monthly averaged values and add the average air temperature in Seattle to the chart.
x = "Date"
group_col = [pd.Grouper(key=x, freq='MS', sort=True, label='left', closed='left')]

monthly_df = (daily_eoy_df.groupby(group_col).agg({y_1: 'mean'}))

monthly_df = monthly_df.reset_index()
y_2 = 'Temperature'

monthly_df[y_2] = [8, 10, 12, 15, 18, 20, 23, 23, 20, 15, 10, 7, 8] #Source: NOAA


fig = plot_multi(monthly_df, x, y_1, y_2)
py.plot(fig, filename='monthly_price_and_temperature', auto_open = False)